## What is Screen-Scraping?


Screenscraping refers to the process of automatically extracting data from web pages. 

A typical screenscraping program:
- download a webpage in HTML format
- finds some piece of desired information 
- places that information in a convenient format

Screenscraping can also be used to download other types of content as well, however, such as audio-visual content.

## Reading content from a web-page in Python 

Let's say we wanted to parse a table from this web page:
[https://en.wikipedia.org/wiki/List_of_countries_by_social_welfare_spending](https://en.wikipedia.org/wiki/List_of_countries_by_social_welfare_spending)


First, we would import a couple of useful packages. 

In [24]:
from bs4 import BeautifulSoup ##A package to work with HTML data
import requests #A package to make HTTP requests
from io import StringIO


Then, we would request the page from the internet using the `requests` package, and parse the HTML content using `Beautifulsoup`. We store the content of the page in a variable named _soup_.

In [4]:
LINK = "https://en.wikipedia.org/wiki/List_of_countries_by_social_welfare_spending"

headers = {
    "User-Agent": "DTU-Compute-Research-Bot/1.0"
}

r = requests.get(LINK, headers=headers)
soup = BeautifulSoup(r.content, "html.parser")


To find our table, we open the elements panel from our browser (Command + Shift + C on Chrome).
Scrolling through the elements, we find that the our table is stored in a _&lt;table&gt;_ element of class _wikitable_. 
Each row in the table is stored in a _&lt;tr&gt;_ table element.
The row's cells can then be established using a mix of _&lt;td&gt;_ (data cell) and _&lt;th&gt;_ (header cell) elements.



We use the command ``find`` to find the first element of this kind within _soup_. 

Then, we use the command ``find_all`` to find all the rows (table row elements _&lt;tr&gt;_) in our table. We loop through the rows and use ``find_all`` to get the headers (table header elements _&lt;th&gt;_) and the data (table data elements _&lt;td&gt;_). 

In [12]:

table = soup.find("table",{"class":"wikitable"})
table_rows = table.find_all("tr")

#HERE I GET THE HEADER
clean = lambda th: th.get_text(" ", strip=True)

# Row 0
ths = table_rows[0].find_all("th")
row0 = [clean(th) for th in ths for _ in range(int(th.get("colspan", 1)))]

# Row 1
ths_2 = table_rows[1].find_all("th")
row1 = [clean(th) for th in ths_2 for _ in range(int(th.get("colspan", 1)))]

# Headers from row0 that span into row1 (e.g., Country)
carry = [clean(th) for th in ths if int(th.get("rowspan", 1)) == 2]

# Full second-row header (Country placeholder + the rest)
row1_full = carry + row1

headers = [(i+"-"+j)  if i!=j else i for (i,j) in zip(row0,row1_full)]


#HERE I GET THE ROWS
rows = []
for tr in table_rows[2:]:
    tds = tr.find_all('td')
    row = [td.text.replace("\n","") for td in tds]
    rows.append(row)           
         

In [13]:
import pandas as pd
pd.DataFrame(rows, columns=headers)

,Country,% of GDP-Public,% of GDP-Net total,Per capita ($)-Constant,Per capita ($)-Current
0,Australia,17.1,22.9,"8,659","12,246"
1,Austria,31.6,29.4,"16,364","20,349"
2,Belgium,28.6,26.7,"14,138","18,007"
3,Bulgaria,20.5,—,"4,622","6,054"
4,Canada,19.3,—,"8,967","11,974"
5,Chile,12.9,20.9,"3,019","4,217"
6,Colombia,14.1,17.4,"2,091","3,093"
7,Costa Rica,12.6,13.0,"2,548","3,291"
8,Croatia,22.5,—,"6,690","8,298"
9,Czech Republic,22.1,20.8,"8,607","11,108"


There is a way to get a table from an html page automatically using Pandas (using ``pd.read_html``). 
However, this will not be useful if you need to parse content other than tables.

In [25]:
tables = pd.read_html(StringIO(r.text), flavor="lxml")
tables[1]

Country % of GDP           Per capita ($)        
           Country   Public Net total       Constant Current
0        Australia     17.1      22.9           8659   12246
1          Austria     31.6      29.4          16364   20349
2          Belgium     28.6      26.7          14138   18007
3         Bulgaria     20.5         —           4622    6054
4           Canada     19.3         —           8967   11974
5            Chile     12.9      20.9           3019    4217
6         Colombia     14.1      17.4           2091    3093
7       Costa Rica     12.6      13.0           2548    3291
8          Croatia     22.5         —           6690    8298
9   Czech Republic     22.1      20.8           8607   11108
10         Denmark     26.4      24.4          15478   19768
11         Estonia     17.2      16.1           6819    8585
12         Finland     31.4      25.6          14110   17788
13          France     30.6      32.0          13588   18147
14         Germany     27.9      28.5          14678   18172
15          Greece     23.7      22.5           7455    8795
16         Hungary     18.0      16.5           6117    7216
17         Iceland     19.2      24.5          11842   14504
18         Ireland     14.4      13.7          14059   15837
19          Israel     16.3      18.6           6912    8764
20           Italy     27.6      26.5          11746   15056
21           Japan     24.7      26.9          10497   11856
22          Latvia     19.7      19.1           6327    7739
23       Lithuania     16.9      17.7           7244    8964
24      Luxembourg     24.0      17.7          24288   29667
25          Mexico     10.0       9.4           1964    2559
26     Netherlands     18.9      27.0          10942   13694
27     New Zealand     24.6      22.2          10258   12792
28          Norway     24.1      20.6          14758   20839
29            Peru      7.5         —            928    1285
30          Poland     23.1      18.4           7747    9258
31        Portugal     24.1      24.7           8117    9867
32         Romania     17.6         —           4911    6686
33        Slovakia     19.3      18.4           6555    7301
34        Slovenia     23.0      22.5           9159   11353
35     South Korea     15.3         —           7668    9009
36           Spain     25.9      26.6          10310   12547
37          Sweden     26.1      23.9          13655   16615
38     Switzerland     16.0      26.1          12055   14281
39          Turkey     10.4      11.1           3474    3492
40  United Kingdom     23.0      26.0          10247   13184
41   United States     19.8      33.2          12284   15293
42            OECD     21.2      22.6           9969   12198